****Objetivo:****

La franquicia de Pokemón ha sido presentada a generaciones a travez de los videojuegos y el anime. Yo recuerdo, con entrañable cariño, a Charmander. Ahora quisiera conocer más pokemones como este, y verificar si este o sus respectivas evoluciones podrían estar al nivel de un pokemon legendario.

In [ ]:
import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
import scipy.stats                # statistics
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage

import os
print(os.listdir("../input"))

In [ ]:
df = pd.read_csv("../input/Pokemon.csv")
print(df.info())
print(df.shape)


**Exploración de los datos:**

Veamos que información hay en la base.

In [ ]:
df.head(12)


In [ ]:
df.iloc[:,~df.columns.isin(['#','name','Type1','Type1','Legendary'])].describe()

Para el caso de esta base no es necesario completar la información existente. Vemos que está la columna total, la cual para el caso de los cluster no se usaría.

Pasemos a observar la correlación entre las variables existentes.

In [ ]:
corr=df.iloc[:,1:13].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(2, 110, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.6, cbar_kws={"shrink": .6})

De esto hay varias cosas que decir.

*  Podemos apreciar que la generación a la que pertenece el Pokemón no se relaciona en nada con sus habilidades (Descartaremos esta en el porceso de Clustering). 
*  Claramente el poder total se relaciona con todos todas las habilidades (Descartaremos esta en el porceso de Clustering). 
*  Lo curioso es ver como el poder del Special Attack esta muy relacionado con la clasificación a legendario (Veremos si el proceso de Clustering termina agurpando a los legendarios), y a su vez con su velocidad y el poder del Special Defense.

Grafiquemos por separado para confirmar que no hay dos habilidades fuertemente relacionadas. 

In [ ]:
#Quitando las columnas de clasificación y el Total (ya que no me interesa estudiar el comportamiento de esta bajo las otras habilidades) 
ds=df.drop(df.columns[[0,4,11,12]], axis=1)
sns.pairplot(ds)

**Estandarización de Datos:**

Solo estandarizaré las variables a tener en cuenta en el proceso de clustering, es decir el poder de cada una de las habilidades básicas y especiales que tiene el pokemon.

In [ ]:
df_scale = df.copy()
scaler = preprocessing.StandardScaler()
columns =df.columns[5:11]
df_scale[columns] = scaler.fit_transform(df_scale[columns])
df_scale.head(10)

Charmander está por debajo de la media en todas su habilidades, lo cual es normal por no estar evolucionado. Por otro lado Charizard esta por encima de la media, ¿Será lo suficiente para enfrentarse a un Legendario?

Por el momento podemos verificar si se encuentra en el mismo grupo de los legendarios. Probemos primero con K-means.

**Clustering:**

*1.  K-means: *

Como vimos en clase, el primer paso será encontrar el número de cluster que minimiza la varianza.

In [ ]:
#Elbow graph
ks = range(1, 10)
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(df_scale.iloc[:,5:])
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o')
plt.xlabel('Cantidad de clusters, k')
plt.ylabel('Inercia')
plt.xticks(ks)
plt.show()

De esta manera intentaré primero con *k=4*. Así,

In [ ]:
model = KMeans(n_clusters=4)
model.fit(df_scale.iloc[:,5:10])
df_scale['cluster'] = model.predict(df_scale.iloc[:,5:10])
df_scale.head(20)

Veamos las media de las habilidades de cada cluster y compararlos.

In [ ]:
df_scale.iloc[:,~df_scale.columns.isin(['#','name','Type1','Type1','Legendary'])].groupby('cluster').mean()

Aparentemente los cluster resultantes son los siguientes:

* **Cluster 3**: Pokemon con habilidades por debajo de la media, la mayoria de ellos son no evolucionados (Pichu, Charmander...) pero hay algunas excepciones del siguiente nivel de evolución como Pikachu. En general, su mayor habilidad es Speed, pero no está por encima del promedio.
* **Cluster 1**: Son todos los pokemon cuyas caracteristicas más fuertes son Hit Point (HP) y Speed, pero pueden ser derrotados con mayor facilidad por tener una defensa por debajo del promedio. En este grupo se encuentran la mayoria de la primera evolucion  (Charmeleon, Wartortle, Ivysaur).
* **Cluster 2**: Son los Pokemon destacados por su poder defensa (La defensa media más alta de los cluster) aunque no tienen gran velocidad. En esta agrupación se encuentran algunos pokemon de la segunda evolución (Blastoise), y también se encuentra un Legendario.
* **Cluster 0**: ***Son los pokemon más destacados***. Aquí se encuntran la gran Mayoría de los legendarios y algunos de las ultimas evoluciones de los otros pokemon. En general todas sus habilidades están por encima del promedio, particularmente sus Special Atack.

Curiosamente, en este último cluster se encuentran 3 de las 4 evoluciones de Charmander, los cuales tienen 1 o 2 habilidades por encima de la media.

In [ ]:
# Replicando la grafica de la clase :)
model_pca = PCA()
pca_features = model_pca.fit_transform(df_scale.iloc[:,5:11])
xs = pca_features[:,0]
ys = pca_features[:,1]
sns.scatterplot(x=xs, y=ys, hue="cluster", data=df_scale)

Ahora Generemos el Dendograma y comparemos entre *Jerárquico Complete *y *K-means*

*2. Jerárquico Complete:*

En este caso veamos como se ve el deondograma.

In [ ]:
Z = linkage(df_scale.iloc[:,5:11], 'complete')

plt.figure(figsize=(10, 150))
plt.title('Dendograma jerárquico')
plt.xlabel('Pokemon')
plt.ylabel('Distancia')
dg = df_scale.set_index('Name')


dendrogram(Z, labels=dg.index, leaf_rotation=0, orientation="left", leaf_font_size=12., show_contracted=False)
my_palette = plt.cm.get_cmap("tab10", 4)
df_scale['cluster']=pd.Categorical(df_scale['cluster'])
my_color=df_scale['cluster'].cat.codes
 
# Apply the right color to each label
ax = plt.gca()
xlbls = ax.get_ymajorticklabels()
num=-1
for lbl in xlbls:
    num+=1
    val=my_color[num]
    lbl.set_color(my_palette(val))


Graficamente se puede notar que los grupos no son similares a los formados en el anterior método. Claramente, el método aísla algunos pokemon ¿Qué particularidades tienen? en ninguno de los grupos más pequeños está Charmander o sus evoluciones.
Veamos con más detalle como están compuestos estos grupos.

In [ ]:
from scipy.cluster.hierarchy import fcluster
clusters2 = fcluster(Z, 9,criterion='distance')
df_scale['clusters2'] = clusters2

df_scale.iloc[:,~df_scale.columns.isin(['#','name','Type1','Type1','Legendary'])].groupby('clusters2').mean()


Inicialmente podemos ver que el grupo 1 y 0 tienen el mayor Hit Point y mayor poder de Defensa respectivamente. Por otro lado el grupo 2 y el grupo 3 tienen a los mas fuertes (todas las habilidades por encima de la media) y los más debiles (todas las habilidades por debajo de la media) respectivamente.

Ahora comparemos su composición con *K-means* (columna "cluster")

In [ ]:
ax = sns.countplot(x="clusters2", hue="cluster", data=df_scale)

* El **Grupo 1** es el compuesto por 11 elementos, como dije anteriormente son los que tienen un HP muy alto, por lo cual no es extraño que se componga en su mayoria de los del *Cluster 1* (HP por encima del promedio)  y uno del *Cluster 0* (Los má poderosos).
* El **Grupo 4** solo contiene un elemento del *Cluster 0* el de mayor poder de defensa y mas debil en su ataque.
* El **Grupo 3** contiene los debiles por lo que no es de extrañar que contenga la mayoria de los del *Cluster 3* (donde se ubicaban los no evolucionados en el anterior método)
* El **Grupo 2** es compuesto por la gran mayoría, cuyas habilidades son en pormedio mayores a los del *Grupo 2*, pero sin superar el HP del *Grupo 1* y la defensa del *Grupo 4*. Parece que es el grupo de los "démas", no creo que tengan una característica especial.

En mi opinión este método no ayuda a clasificar en grupos uniformes, este es bastante útil para hallar singularidades en la Base de Datos. Podriamos concentrarnos ahora en los *grupos 2 y 3*.

¿En dónde está ubicado charmander y sus evoluciones? ¿en qué grupo hay más legendarios?

In [ ]:
ax = sns.countplot(x="Legendary", hue="clusters2", data=df_scale)

Los Legendarios están en el *Grupo 2* , excepto por uno en el *Grupo 3* que muy seguramente es el mismo que estaba en el *Cluster 0*. Así que veamos si aquí estan la evoluciones de Charmander.

In [ ]:
df2 = df_scale[df_scale['clusters2'] == 2]

df2.head(10)


Curiosamente en este grupo también se encuentra Charmander. Y como sospechaba no hay una característica particular que destaque al grupo. Son má fuertes que el *Grupo 3*, nada más.
Veamos la misma grafica que se corrió por Cluster de *K-means*

In [ ]:
# Replicando la grafica de la clase :)
model_pca = PCA()
pca_features = model_pca.fit_transform(df_scale.iloc[:,5:11])
xs = pca_features[:,0]
ys = pca_features[:,1]
sns.scatterplot(x=xs, y=ys, hue="clusters2", data=df_scale)

Claramente el *Grupo 2* esta muy disperso. en conclusión, para responder la pregunta acerca de Charmander, es más eficiente basarse en *K-means*. 
